In [1]:
import iris
import pandas as pd


host = "localhost"
# this is the superserver port
port = 32782
namespace = "FHIRSERVER"
user = "_SYSTEM"
password = "SYS"

conn = iris.connect(
    hostname=host,
    port=port,
    namespace=namespace,
    username=user,
    password=password
)

# create a cursor
cursor = conn.cursor()

sql = """
      select * from (
                        select prac._id, prac.Key, GetFHIRPathOne(resource.ResourceString, 'Practitioner.identifier[0].value','Practitioner') As GMP from HSFHIR_X0001_S.Practitioner prac                                           join HSFHIR_X0001_R.Rsrc resource on  resource.ID = prac._id
                        where IsNull(prac.addressCountry,'') <> 'US')
      group by GMP having count (*) > 1
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)
pd.set_option('future.no_silent_downcasting', True)

df

,_id,Key,GMP
0,765722,Practitioner/765722,G0102005
1,765723,Practitioner/765723,G0102926
2,765724,Practitioner/765724,G0105912
3,765725,Practitioner/765725,G0107031
4,765726,Practitioner/765726,G0107725
...,...,...,...
4995,775719,Practitioner/775719,G3297023
4996,775720,Practitioner/775720,G3297054
4997,775721,Practitioner/775721,G3297078
4998,775722,Practitioner/775722,G3297102


In [2]:
import requests

headers = {"Content-Type": "application/fhir+json"}
fhirurl = "http://localhost:32783/fhir/r4"

for gpId in range(0, len(df)):
    id = df.loc[gpId,'_id']
    r = requests.delete(fhirurl+'/Practitioner/'+id, headers=headers)
